In [ ]:
!pip install datasets==2.14.6
!pip install transformers
!pip install evaluate
!pip install --no-cache-dir transformers sentencepiece
!pip install accelerate -U

In [12]:
import torch
from datasets import Dataset, DatasetDict
from datasets import concatenate_datasets
from torch.utils.data import DataLoader
import os
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
from transformers import AutoTokenizer, AutoModelForMultipleChoice, get_scheduler, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import evaluate
import accelerate

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# model_name = "FacebookAI/roberta-large"
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
train_data = np.load('/content/drive/MyDrive/data/SP-train.npy', allow_pickle=True)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [14]:
def convert_to_dataset_type(array):
    df = pd.DataFrame(array.tolist())
    col = ['id','distractor1','distractor2','distractor(unsure)']
    for c in col:
      df[c] = df[c].astype(str)
    df['label'] = df['label'].astype(int)
    data = Dataset.from_pandas(df, split = "train")
    return data

def preprocess_function(examples):
    first_sentences = [[context] * 4 for context in examples["question"]]
    first_sentences = sum(first_sentences, [])
    second_sentences = [item for item in examples["choice_list"]]
    second_sentences = sum(second_sentences, [])
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

In [15]:
train_dataset = convert_to_dataset_type(train_data)
tokenized_train = train_dataset.map(preprocess_function, batched=True)
print(f"Training set size: {len(tokenized_train)}")

Map:   0%|          | 0/507 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Training set size: 507


In [17]:
def split_dataset(dataset):
    df = dataset.to_pandas()
    train_temp, temp_df = train_test_split(df, test_size=0.2, shuffle=True)
    val_df, test_df = train_test_split(temp_df, test_size=0.5, shuffle=True)
    train_dataset = Dataset.from_pandas(train_temp)
    val_dataset = Dataset.from_pandas(val_df)
    test_dataset = Dataset.from_pandas(test_df)
    dataset_dict = DatasetDict({
        "train": train_dataset,
        "valid": val_dataset,
        "test": test_dataset
    })
    return dataset_dict

my_dataset = split_dataset(tokenized_train)
print("Training dataset size:", len(my_dataset['train']))
print("Validation dataset size:", len(my_dataset['valid']))
print("Testing dataset size:", len(my_dataset['test']))

Training dataset size: 405
Validation dataset size: 51
Testing dataset size: 51


In [18]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [19]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [20]:
def get_final_dataset(dataset):
    tokenized_dataset = dataset.rename_column("label", "labels")
    tokenized_dataset = tokenized_dataset.remove_columns(['id', 'question', 'answer', 'distractor1', 'distractor2', 'distractor(unsure)', 'choice_list', 'choice_order'])
    tokenized_dataset.set_format("torch")
    return tokenized_dataset

tokenized_datasets = get_final_dataset(my_dataset)
os.environ["WANDB_DISABLED"] = "true"

model = AutoModelForMultipleChoice.from_pretrained(model_name, ignore_mismatched_sizes=True)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model.to(device)

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda


DebertaV2ForMultipleChoice(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
 

In [21]:
batch_size = 4
lr = 5e-5
num_epochs = 5

num_training_steps = (len(my_dataset["train"]) // batch_size) * num_epochs
batches_per_epoch = len(my_dataset["train"]) // batch_size

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [22]:
training_args = TrainingArguments(
    output_dir = "./output",
    evaluation_strategy="steps",
    eval_steps=20,
    logging_steps=20,
    logging_strategy="steps",
    learning_rate=lr,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    report_to=None,
    save_strategy="no"
    )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [24]:
print(f'Training {model_name}')
train_result = trainer.train()

Training microsoft/deberta-v3-base


You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
20,1.309300,1.112168,0.666667
40,0.991300,0.603915,0.784314
60,0.706100,0.823974,0.686275
80,0.513000,0.603193,0.803922
100,0.448800,0.402704,0.882353
120,0.174200,0.374332,0.862745
140,0.242100,0.320326,0.862745
160,0.148100,0.308022,0.921569
180,0.220600,0.224329,0.901961
200,0.148200,0.433391,0.882353


In [25]:
metrics = train_result.metrics
trainer.save_metrics("train", metrics)
trainer.save_state()

In [26]:
def evaluate_accuracy(dataset, pred_list=False):
    total_answers = 0
    correct_answers = 0
    predictions = []

    model.eval()

    for i in dataset:
        prompt = i['question'].strip()
        candidates = i['choice_list']
        true_label_original = i['label']
        candidate_1, candidate_2, candidate_3, candidate_4 = candidates[0].strip(), candidates[1].strip(), candidates[2].strip(), candidates[3].strip()

        inputs = tokenizer([[prompt, candidate_1], [prompt, candidate_2], [prompt, candidate_3], [prompt, candidate_4]],
                           return_tensors="pt", padding=True).to("cuda")

        labels = torch.tensor(true_label_original).unsqueeze(0).to("cuda")  # Batch size 1

        # Pass the input through the model to obtain predictions
        with torch.no_grad():
            outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)

        logits = outputs.logits
        predicted_class = logits.argmax().item()

        predictions.append(predicted_class)

        if predicted_class == true_label_original:
            correct_answers += 1
        total_answers += 1

    # Calculate accuracy
    accuracy = correct_answers / total_answers

    # Round accuracy to three decimal places
    rounded_accuracy = round(accuracy, 3)
    print("Accuracy is", rounded_accuracy)

    if pred_list:
        return rounded_accuracy, predictions
    return rounded_accuracy

In [27]:
acc = evaluate_accuracy(my_dataset['test'])

Accuracy is 0.961
